In [1]:
!pip install google-cloud-storage


In [5]:
import sys
import os

# Get the absolute path of the 'app' folder
sys.path.append(os.path.abspath('../app'))

# Now import Frame from frame.py
from frame import Frame

# Create an instance and call dataset()
data = Frame()
df = data.dataset()


Fetching and Merging...
Merging completed successfully!
Started prepocessing....
Completed prepocessing.


/home/sb/lewagon_london/project_lewagon/stock-stat-replica/app/frame.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest.dropna(inplace=True)


In [10]:
df['date'].value_counts(ascending=False)

date
2025-03-07    1
2022-07-15    1
2022-07-18    1
2022-07-19    1
2022-07-20    1
             ..
2022-08-08    1
2022-08-05    1
2022-08-04    1
2022-08-03    1
2022-08-02    1
Name: count, Length: 668, dtype: int64

In [27]:
from google.cloud import storage
import pandas as pd
import io
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [3]:
from google.colab import files

uploaded = files.upload()  #upload credentials file

Saving lewagon-statistical-arbitrage-ae470f7dcd48.json to lewagon-statistical-arbitrage-ae470f7dcd48 (2).json


In [4]:
filename = list(uploaded.keys())[0]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/" + filename

In [5]:
client = storage.Client()
bucket = client.get_bucket('stat_arb')


In [6]:
rename_dict = {
                    'Unnamed: 0': 'date',
                    '1. open': 'open',
                    '2. high': 'high',
                    '3. low': 'low',
                    '4. close': 'close',
                    '5. volume': 'volume'
                }

In [55]:
folder_prefix = "FTSE_100/"
blobs = bucket.list_blobs(prefix=folder_prefix)
dataframes = []

for blob in blobs:
    if blob.name.endswith('.csv'):
        content = blob.download_as_string()
        if content.strip():
            try:
                df = pd.read_csv(io.StringIO(content.decode('utf-8')), sep=None, engine='python')

                df = df.rename(columns=rename_dict)
                expected_columns = rename_dict.values()
                df = df[[col for col in df.columns if col in expected_columns]]
                stk=blob.name.split('/')[1]
                df['source_file']=stk.split('.')[0]
                dataframes.append(df)

            except pd.errors.EmptyDataError:
                print(f"Skipping empty file: {blob.name}")
        else:
            print(f"Skipping empty file: {blob.name}")

if dataframes:
    merged_df = pd.concat(dataframes, ignore_index=True)
    print("Merging completed successfully!")
else:
    print("No valid CSV files found.")

merged_df

Skipping empty file: FTSE_100/test.csv
Merging completed successfully!


,date,open,high,low,close,volume,source_file
0,2025-03-07,142.4,145.4,142.09,145.4,3523562.0,AAF
1,2025-03-06,143.9,143.9,140.10,142.7,3290372.0,AAF
2,2025-03-05,142.8,143.3,140.80,142.2,3266502.0,AAF
3,2025-03-04,142.4,145.0,139.80,141.3,2437636.0,AAF
4,2025-03-03,143.1,143.6,140.00,143.3,2425338.0,AAF
...,...,...,...,...,...,...,...
471419,2005-01-10,556.5,563.0,550.50,561.5,17179131.0,WPP
471420,2005-01-07,558.0,566.0,554.50,556.0,13183020.0,WPP
471421,2005-01-06,560.0,566.5,559.00,560.5,6084216.0,WPP
471422,2005-01-05,560.0,567.0,558.50,560.5,10175660.0,WPP


In [60]:
merged_df['date'] = pd.to_datetime(merged_df['date'],format='mixed')


In [62]:
df_modified = merged_df[['date', 'source_file', 'open']].copy()
df_modified['source_file'] = df_modified['source_file'].str.split('/').str[-1].str.replace('.csv', '')
df_pivoted = df_modified.pivot_table(index='date', columns='source_file', values='open')
df_pivoted.reset_index(inplace=True)

df_pivoted

source_file,date,AAF,AAL,ABF,ADM,AHT,ALW,ANTO,AUTO,AV,...,STJ,SVT,TSCO,TW,ULVR,UTG,UU,VOD,WEIR,WPP
0,2005-01-03,NaN,NaN,NaN,NaN,NaN,2728.0,NaN,NaN,NaN,...,214.7179,NaN,NaN,NaN,511.50,NaN,NaN,80.3587,NaN,NaN
1,2005-01-04,NaN,1143.3125,775.0,323.00,77.5000,2740.0,1111.0,NaN,631.9998,...,212.6923,796.9373,319.2501,205.1379,514.25,284.5826,619.6548,91.7897,324.00,566.0
2,2005-01-05,NaN,1124.6309,781.5,327.00,81.5000,2711.0,1080.0,NaN,628.4998,...,219.7821,799.8025,315.5001,212.1592,512.50,286.7740,626.0735,92.1310,330.00,560.0
3,2005-01-06,NaN,1134.9058,774.0,334.75,80.5000,2721.0,1076.0,NaN,639.9998,...,217.5032,803.0770,315.0001,201.3425,514.75,284.5826,627.0610,92.4722,325.00,560.0
4,2005-01-07,NaN,1147.9829,770.0,338.00,82.0001,2745.0,1094.0,NaN,640.9998,...,220.7949,789.5696,317.7501,202.6709,511.25,282.2111,631.5048,92.4722,329.75,558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5115,2025-03-03,143.1,2335.5000,1895.0,2871.00,4809.0000,1268.0,1738.5,774.4,542.4000,...,1061.0000,2477.0000,380.0000,112.8000,4486.00,829.0000,972.4000,70.0600,2438.00,635.4
5116,2025-03-04,142.4,2303.0000,1884.5,2900.00,4687.0000,1256.0,1751.0,785.0,536.6000,...,1095.0000,2438.0000,377.2000,110.4500,4569.00,826.5000,965.6000,70.1000,2452.00,625.8
5117,2025-03-05,142.8,2375.0000,1871.0,2907.00,4425.0000,1222.0,1771.0,779.2,535.6000,...,1065.0000,2420.0000,386.2000,111.6500,4565.00,824.5000,964.6000,69.5600,2420.00,623.0
5118,2025-03-06,143.9,2418.0000,1866.5,3117.00,4588.0000,1232.0,1842.5,777.0,542.6000,...,1049.0000,2385.0000,381.4000,111.2500,4520.00,810.0000,940.4000,70.0800,2466.00,616.8


In [71]:
df_pivoted[df_pivoted['AAF'].isna()]

source_file,date,AAF,AAL,ABF,ADM,AHT,ALW,ANTO,AUTO,AV,...,STJ,SVT,TSCO,TW,ULVR,UTG,UU,VOD,WEIR,WPP
0,2005-01-03,NaN,NaN,NaN,NaN,NaN,2728.0,NaN,NaN,NaN,...,214.7179,NaN,NaN,NaN,511.50,NaN,NaN,80.3587,NaN,NaN
1,2005-01-04,NaN,1143.3125,775.0,323.00,77.5000,2740.0,1111.0,NaN,631.9998,...,212.6923,796.9373,319.2501,205.1379,514.25,284.5826,619.6548,91.7897,324.00,566.0
2,2005-01-05,NaN,1124.6309,781.5,327.00,81.5000,2711.0,1080.0,NaN,628.4998,...,219.7821,799.8025,315.5001,212.1592,512.50,286.7740,626.0735,92.1310,330.00,560.0
3,2005-01-06,NaN,1134.9058,774.0,334.75,80.5000,2721.0,1076.0,NaN,639.9998,...,217.5032,803.0770,315.0001,201.3425,514.75,284.5826,627.0610,92.4722,325.00,560.0
4,2005-01-07,NaN,1147.9829,770.0,338.00,82.0001,2745.0,1094.0,NaN,640.9998,...,220.7949,789.5696,317.7501,202.6709,511.25,282.2111,631.5048,92.4722,329.75,558.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,2019-06-27,NaN,2197.6527,2430.0,2191.00,2224.0000,790.0,910.2,564.0,414.1998,...,1080.5000,2059.0000,224.3001,155.0500,4887.50,978.5000,798.4000,126.7800,1500.50,980.2
3906,2020-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4651,2023-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4666,2023-05-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df_latest=df_pivoted[df_pivoted['date']>'2022-01-31']

In [102]:
df_latest.isna().sum()

,0
source_file,
date,0
AAF,3
AAL,3
ABF,3
ADM,3
...,...
UTG,3
UU,3
VOD,3


In [103]:
df_latest.dropna(inplace=True)

<ipython-input-103-38a5827a54aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest.dropna(inplace=True)


In [104]:
df_numeric = df_latest.drop(columns=['date'])


In [105]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

In [106]:
pca = PCA(n_components=0.95)
df_pca = pca.fit_transform(df_scaled)

In [107]:
df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])


In [109]:
df_pca['date'] = df_latest['date'].values


In [110]:
df_pca

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,date
0,-9.934486,4.783074,-2.008245,-5.094189,-6.151302,-1.043500,1.179703,0.070893,0.924902,-0.567824,-0.104906,1.186363,0.783230,2022-07-15
1,-9.394046,5.846684,-1.477654,-4.814168,-5.223393,-0.517745,1.807719,0.023307,1.518427,-0.053822,-0.201370,1.097387,0.687002,2022-07-18
2,-9.496939,6.146037,-0.868169,-5.214040,-5.304921,-0.288722,1.987788,-0.097964,1.550844,-0.160769,0.713017,1.909640,0.258940,2022-07-19
3,-8.925005,7.645625,-0.539084,-4.955993,-4.750791,0.332712,2.426275,-0.097143,1.773227,-0.036723,0.641246,2.121806,0.092457,2022-07-20
4,-8.825943,7.492370,-0.432681,-5.226540,-5.082073,0.672217,1.950272,0.035883,2.091026,-0.168914,0.327025,2.088623,0.333566,2022-07-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,13.167511,-0.998679,-7.795343,-0.142448,0.521090,2.988276,4.316584,-0.201020,-1.358177,0.844784,1.265655,-0.590691,1.270639,2025-03-03
664,13.142652,-1.267232,-7.959212,-0.191148,0.524153,2.822163,4.611795,-0.575691,-1.082489,1.142836,1.433263,-1.010294,1.475423,2025-03-04
665,12.768656,-1.497292,-8.189921,-0.110859,0.656913,2.621967,4.753752,-0.242873,-1.339772,1.287965,1.464527,-0.966022,1.531170,2025-03-05
666,12.856982,-1.583376,-7.890534,-0.223034,1.430963,2.762648,4.943234,-0.556575,-1.172682,1.403105,1.238190,-1.426207,2.020716,2025-03-06


In [111]:
print("Explained Variance Ratio:", pca.explained_variance_ratio_)


Explained Variance Ratio: [0.47344233 0.12771484 0.12177298 0.06782373 0.04573221 0.03420741
 0.02278499 0.01592695 0.01172726 0.01082689 0.00861292 0.00666667
 0.00550069]
